# RcloneFS

This object wraps an `rclone` object from the __rclone-python__ project, which in turn wraps the interface to the __rclone__ project.

`RcloneFS` implements `pyfilesystem2` methods.


#### Development status

The first methods implemented are:

- `listdir()`
- `getinfo()`

These provide basic browsing functionality.

This is for free from the `FS` baseclass:

- `isdir()`

#### Timeline

I'm doing this as a component in another project. I was motivated to enable those features so I can get some basic browsability. Now that they are done, I'm moving back to the other project to complete some more code there, and then returning to flesh the rest of this out. Could be days, weeks...months even. Who knows! But seriously. Next week. --Raygan

In [163]:
# fs.rclonefs
# rclonefs.py

# Let's get this party started.

from fs.base import FS
from fs.info import Info
from fs.errors import FSError
from fs.errors import CreateFailed
from fs.errors import ResourceNotFound
from fs.errors import DirectoryExpected
from fs.enums import ResourceType
from fs.path import basename
from fs.path import dirname
from fs.path import abspath
from fs.path import normpath
import json

# this is our rclone_handler
from rclone_python import rclone


def helper_type_from_mime_type(mime_type):
    """Turn mime type into ResourceType.
    (Map to files or directories.)
    These are just for starters...
    """
    if mime_type == 'inode/directory':
        return int(ResourceType.directory)
    elif mime_type == 'inode/file':
        return int(ResourceType.file)
    elif mime_type == 'application/octet-stream':
        return int(ResourceType.file)
    else:
        return int(ResourceType.unknown)

class RcloneFS(FS):
    """Provide a pyfilesystem interface for an rclone remote.
    """

    def __init__(self, rclone_remote_name, rclone_handler=rclone):
        """
        Arguments:
            self
            rclone_remote_name
            rclone_handler
        Raises:
            fs.errors.CreateFailed
        """
        
        self._rclone = rclone_handler
        self._remote_name = rclone_remote_name if rclone_remote_name.endswith(':') else rclone_remote_name + ':'
        if not self._rclone.is_installed:
            raise CreateFailed("Requires rclone. For installation see https://rclone.org/install/")
        if not self._rclone.check_remote_existing(rclone_remote_name):
            raise CreateFailed('Please use the name of an existing remote.')
        super().__init__()

    def getinfo(self, path, namespaces=None):
        # type: (Text, Optional[Collection[Text]]) -> Info
        """
        Arguments:
            path (str): 
            namespaces (list, optional): "basic" always included
        Returns:
            ~fs.info.Info: resource information object.

        Raises:
            fs.errors.ResourceNotFound: If ``path`` does not exist.
        
        """
        _abspath = abspath(normpath(path))
        _dirname = dirname(_abspath)
        _basename = basename(_abspath)
            
        with self._lock:
            
            if _abspath == '/':
                # it's fakeroot time.
                about = self._rclone.about(self._remote_name)
                raw_info = {
                    'basic': {
                        'name': '/',
                        'is_dir': True
                    },
                    'details': {
                        'accessed': None,
                        'created': None,
                        'metadata_changed': None,
                        'modified': None,
                        'size': about['used'],
                        'type': 1
                    },
                    'rclone.about': about
                }
                return Info(raw_info)

            try:
                _dir = self._rclone.ls(self._remote_name + _dirname)
                _e = [e for e in _dir if e["Name"] == _basename][0]
                _type = helper_type_from_mime_type(_e["MimeType"])
                raw_info = {
                    'basic': {
                        'name': _e["Name"],
                        'is_dir': _e["IsDir"]
                    },
                    'details': {
                        'accessed': None,
                        'created': None,
                        'metadata_changed': None,
                        'modified': _e['ModTime'],
                        'size': _e['Size'],
                        'type': _type
                    }
                }
            except Exception as e:
                raise e
            return Info(raw_info)

    # note: the default implementation
    #   of `isdir()` uses the results of
    #   get info.

    def listdir(self, path):
        # type: (Text) -> List[Text]
        """Get a list of the resource names in a directory.

        This method will return a list of the resources in a directory.
        Resources are defined in `~fs.enums.ResourceType`.

        Arguments:
            path (str): A path to a directory on the filesystem.
            
        Returns:
            list: list of names, relative to ``path``.
        
        Raises:
            fs.errors.DirectoryExpected: If ``path`` is not a directory.
            fs.errors.ResourceNotFound: If ``path`` does not exist.
    
        """
        
        _abspath = abspath(normpath(path))
        _dirname = dirname(_abspath)
        _basename = basename(_abspath)
        
        with self._lock:
            try:
                if self.isdir(_abspath):
                    res = self._rclone.ls(self._remote_name + _abspath)
                    return [ e['Name'] for e in res ]
                else:
                    raise DirectoryExpected(path)
            except Exception as e:
                if "directory not found" in str(e):
                    raise ResourceNotFound(path)
                else:
                    raise
                
    
    def makedir(self):
        pass

    def openbin(self):
        pass

    def remove(self):
        pass

    def removedir(self):
        pass

    def setinfo(self):
        pass

In [172]:
from makepy import makepy

In [174]:
makepy('rclonefs')

2024/07/01 00:43:42 rclonefs


In [8]:
#%%sh
#. ../../_venv/bin/activate
#python3.10 -m pip install --upgrade pip
#python3.10 -m pip install rclone-python==0.1.12

In [1]:
import sys
print(sys.path)

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/raygan/Cosms/Dboy/Laydbug/dev/fs.rclonefs/_venv/lib/python3.10/site-packages']


In [164]:
rcfs = RcloneFS('dropbox')


In [168]:
res = rcfs.listdir('/Camera Uploads')

In [169]:
res

['2020-05-27 06.31.51.heic',
 '2020-05-27 06.32.03.heic',
 '2020-05-27 06.32.07.heic',
 '2020-05-29 22.22.16.heic',
 '2020-05-30 09.26.16-1.heic',
 '2020-05-30 09.26.16.heic',
 '2020-05-30 09.26.27-1.heic',
 '2020-05-30 09.26.27.heic',
 '2020-05-30 09.26.32-1.heic',
 '2020-05-30 09.26.32.heic',
 '2020-05-30 09.26.35-1.heic',
 '2020-05-30 09.26.35.heic',
 '2020-05-30 10.39.45.jpg',
 '2020-05-30 10.39.49.jpg',
 '2020-05-30 11.03.28.heic',
 '2020-05-30 11.45.15.jpg',
 '2020-05-30 11.45.16-1-1.jpg',
 '2020-05-30 11.45.16-1.jpg',
 '2020-05-30 11.45.16.jpg',
 '2020-05-30 11.45.17.jpg',
 '2020-05-30 11.45.18-1-1.jpg',
 '2020-05-30 11.45.18-1.jpg',
 '2020-05-30 11.45.18.jpg',
 '2020-05-30 11.45.19-1.jpg',
 '2020-05-30 11.45.19.jpg',
 '2020-05-30 11.45.20.jpg',
 '2020-05-30 11.45.21-1.jpg',
 '2020-05-30 11.45.21.jpg',
 '2020-05-30 11.45.22-1.jpg',
 '2020-05-30 11.45.23-1.jpg',
 '2020-05-30 11.45.23.jpg',
 '2020-05-30 11.45.24.jpg',
 '2020-05-30 11.45.27.jpg',
 '2020-05-30 11.45.29.jpg',
 '2020-

In [171]:
rcfs.getinfo('/Camera Uploads/2020-08-12 11.49.29.heic').size

830756

In [157]:
rcfs.isdir('/Camera Uploads')

True

In [158]:
rcfs.getinfo('//').raw['rclone.about']

{'total': 5637144576, 'used': 8378316560, 'free': -2741171984}

In [33]:
## Test init

In [35]:
testfs = RcloneFS('dropbox:')

In [36]:
failfs = RcloneFS('dripbox:')

CreateFailed: Please use the name of an existing remote.